# Retrainer

In [ ]:
include("src/Retrainer.jl")
using .Retrainer

using Plots

Firstly, you need to set up parameters essential for re-training process:

* Teacher Model Parameters:
  - `file_path` – this is the path to teacher model.

* Student Model Parameters:
  - `file_name` – this is the file name that student model would be saved in.
  - `structure` – this is structure of layers and neurons for student model.
  - `activations` – this is the structure of activation functions between layers.
  - `use_bias` – this is parameter that can turn on/turn off biases for layers in student model.
  - `precision` – this parameter set the precision type for student model. It could be: `f64`, `f32` or `f16`.

* Training Parameters:
  - `epochs` – this is the number of training epochs.
  - `learning_rate` – this is learning rate that would be used for `Adam` optimizer.
  - `input_scale_coef` - this coefficient is scale coefficient for input data. Input data initially just random generated in interval $[0, 1)$, after that it would be multiplied by `input_scale_coef`.

* Testing Parameters:
  - `samples_num` – this is amount of samples that would be used for calculating accuracy.

In [ ]:
# [teacher_model]
file_path = "example-methanol-model.bson"

# [student_model]
file_name = "retrained-model.bson"
structure = [8, 10, 8, 1]
activations = ["identity", "relu", "identity"]
use_bias = true
precision = "f64"

# [training]
epochs = 1000000
learning_rate = 0.001
input_scale_coef = 16

# [testing]
samples_num = 100000;

In [ ]:
student_params = NeuralNetParams(structure, activations, use_bias, precision)
training_params = TrainingParams(epochs, learning_rate, input_scale_coef)
testing_params = TestingParams(input_scale_coef, samples_num);

## Initial training

This procedure create student model and train it to reproduce results of teacher model.

In [ ]:
student_model, teacher_model, losses = run_initial_training(student_params, training_params,
    testing_params, file_path, file_name);

In [ ]:
plot_loss_function(losses)

### Heatmaps of weights of Teacher Model

In [ ]:
plot_model_parameters(teacher_model)

### Heatmaps of weights of Student Model

In [ ]:
plot_model_parameters(student_model)

## Fine tuning of the model

After training the model, usually it is required to train with another set of parameters. It means smaller `learning_rate` very often.

In [ ]:
student_file_path = "retrained-model.bson"
output_file_name = "fine-tuned-model.bson"
teacher_file_path = "example-methanol-model.bson"

# [training]
epochs = 2000000
learning_rate = 0.0001
input_scale_coef = 2

# [testing]
samples_num = 100000

fine_tuning_training_params = TrainingParams(epochs, learning_rate, input_scale_coef)
fine_tuning_testing_params = TestingParams(input_scale_coef, samples_num);

In [ ]:
student_model, teacher_model, losses = run_fine_tuning(student_file_path, output_file_name, teacher_file_path,
    fine_tuning_training_params, fine_tuning_testing_params);

In [ ]:
plot_loss_function(losses)

### Heatmaps of weights of Teacher Model

In [ ]:
plot_model_parameters(teacher_model)

### Heatmaps of weights of Student Model

In [ ]:
plot_model_parameters(student_model)